# print matches of a regex *finditer()*

print the results of substring search in a string.
using regex for the search.

In [ ]:
import re
pattern = re.compile(r'find_me')

## the *print_matches* code

support various ways to print the matches.
start with redundant code.

this is hard to read.
it takes the longest time to see the *structure of the code*.

In [ ]:
class esc_high_colors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class esc_std_colors:
    HEADER = '\033[35m'
    OKBLUE = '\033[34m'
    OKGREEN = '\033[32m'
    WARNING = '\033[33m'
    FAIL = '\033[31m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def p_groups(m):
    print m.group(), (m.start(), m.end())
    for ix, g in enumerate(m.groups()):
        print g, (m.start(ix+1), m.end(ix+1))
        
def p_matches(match_iter):
    for m in match_iter:
        p_groups(m)


def pf_group(group, start, end, esc_color=esc_std_colors.OKGREEN):
    print esc_color + group.rjust(end) + esc_std_colors.ENDC 
    
def pf_groups(m):
    print esc_std_colors.OKBLUE + m.string + esc_std_colors.ENDC

    pf_group(m.group(), m.start(), m.end(), esc_std_colors.WARNING)
    for ix, g in enumerate(m.groups()):
        pf_group(g, m.start(ix+1), m.end(ix+1))
        
def pf_matches(match_iter):
    for m in match_iter:
        pf_groups(m)

In [ ]:
m = pattern.finditer("find_me_at_a_sercret_place_find_me")
pf_matches(m)

In [ ]:
m = pattern.finditer("find_me_at_a_sercret_place_find_me")
p_matches(m)

## refactor *p_matches()* and *pf_matches()*

### wrapper function

call *pf_matches* with a *print_groups* function.
provide a default for *print_groups*.

*p_matches* becomes a wrapper around *pf_matches*.

redundancy in the *matches* functions is gone.
readability might be a tiny bit better.

In [ ]:
def p_matches(match_iter):
    pf_matches(match_iter, p_groups)
        
def pf_matches(match_iter, print_groups=pf_groups):
    for m in match_iter:
        print_groups(m)

In [ ]:
m = pattern.finditer("find_me_at_a_sercret_place_find_me")
pf_matches(m)

introduce a new private function *_p_matches*.

both public functions become a wrapper around *_p_matches*.

readability is better: two functions share code. obviously.

In [ ]:
def _p_matches(match_iter, print_groups):
    for m in match_iter:
        print_groups(m)

def p_matches(match_iter):
    _p_matches(match_iter, p_groups)

def pf_matches(match_iter):
    _p_matches(match_iter, pf_groups)

In [ ]:
m = pattern.finditer("find_me_at_a_sercret_place_find_me")
pf_matches(m)

### factory function

In [ ]:
def create_print(print_groups):
    
    def _p_matches(match_iter):
        for m in match_iter:
            print_groups(m)
    
    return _p_matches

p_matches = create_print(p_groups)
pf_matches = create_print(pf_groups)

In [ ]:
m = pattern.finditer("find_me_at_a_sercret_place_find_me")
pf_matches(m)

again the factory function pattern.

change the code to use the *functools* module.

In [ ]:
import functools as fp

def _p_matches(match_iter, p_func=p_groups):
    for m in match_iter:
        p_func(m)

p_matches = fp.partial(_p_matches, p_func=p_groups)
pf_matches = fp.partial(_p_matches, p_func=pf_groups)

In [ ]:
m = pattern.finditer("find_me_at_a_sercret_place_find_me")
pf_matches(m)

### dependency injection

without the *inject* code, just the *print_matches* code, there is nothing in the code to *install* one of the *p_func* implementations within *p_matches*.

In [ ]:
import inject

def match_printer():
    pass

def create_pf():
    return pf_groups

def create_p():
    return p_groups

def pf_print(binder):
    binder.bind_to_provider(match_printer, create_pf)

inject.clear()
inject.configure(pf_print)

p_func = inject.instance(match_printer)

In [ ]:
def p_matches(match_iter):
    for m in match_iter:
        p_func(m)

In [ ]:
m = pattern.finditer("find_me_at_a_sercret_place_find_me")
p_matches(m)